In [1]:
%env SPARK_VERSION=3.0.0

env: SPARK_VERSION=3.0.0


In [2]:
import pydeequ

from pyspark.sql import SparkSession, Row

# TODO create spark session with jdbc driver path

In [3]:
server_name = "jdbc:sqlserver://host.docker.internal"
database_name = "TRN"
jdbc_url = server_name + ";" + "databaseName=" + database_name + ";trustServerCertificate=True;"

table_name = "hr.jobs"
username = "TestUser3"
password = "TestUser3" # Please specify password here
connection_details = { "user": username, "password": password, "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver", }


spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .config("spark.driver.extraClassPath", "/home/jars/sqlserver/sqlserverjdbc.jar")
    .getOrCreate())

df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=connection_details)

RuntimeError: Java gateway process exited before sending its port number

In [4]:
df.printSchema()

NameError: name 'df' is not defined

In [5]:
df.show(20,False)

NameError: name 'df' is not defined

In [6]:
### Data Analyzers section
# TODO analyze data here
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(ApproxCountDistinct("job_title")) \
                    .addAnalyzer(Completeness("job_id")) \
                    .addAnalyzer(ApproxCountDistinct("job_id")) \
                    .addAnalyzer(Mean("min_salary")) \
                    .addAnalyzer(Mean("max_salary")) \
                    .addAnalyzer(Compliance("min_salary", "min_salary > 0")) \
                    .addAnalyzer(Compliance("max_salary", "max_salary > 0")) \
                    .run()
                    #.addAnalyzer(Compliance("top star_rating", "star_rating >= 4.0")) \
                    #.addAnalyzer(Correlation("total_votes", "star_rating")) \
                    #.addAnalyzer(Correlation("total_votes", "helpful_votes")) \
                    
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

NameError: name 'spark' is not defined

In [7]:
analysisResult_pandas_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult, pandas=True)
analysisResult_pandas_df

NameError: name 'spark' is not defined

In [11]:
### Data profiling section
# TODO profile data here
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print("Column '{}':\n ".format(col) +
        "\tcompleteness: {}\n".format(profile.completeness) +
        "\tapproximate number of distinct values: {}\n".format(profile.approximateNumDistinctValues) +
        "\tdatatype: {}\n".format(profile.dataType) +
        ("\tmean: {}\n".format(profile.mean) if profile.dataType == 'Integral' else '') +
        ("\tmax: {}\n".format(profile.maximum) if profile.dataType == 'Integral' else '' ) +
        ("\tmin: {}\n".format(profile.minimum) if profile.dataType == 'Integral' else '' ) +
        ("\tsum: {}\n".format(profile.sum) if profile.dataType == 'Integral' else '' )
         )

print('Value distribution:\r\n')
for col, profile in result.profiles.items():
    if profile.histogram is not None:
        print('Column {}'.format(col))
        for each in profile.histogram:
            print('\t\tValue {} occured {} times (ratio is {})'.format(each.value, each.count,round(each.ratio,2)))

NameError: name 'spark' is not defined

In [8]:
### Constraint Suggestions section
# TODO find meaninful constraints here
from pydeequ.suggestions import *
from pyspark.sql import Row

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
#print(suggestionResult)
suggdf = spark.createDataFrame(Row(**x) for x in suggestionResult['constraint_suggestions']) #.show(truncate=False)
suggdf.sort("column_name","current_value").select('column_name','current_value','description','code_for_constraint').show(truncate=False)

NameError: name 'spark' is not defined

In [9]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check \
        .hasMin("job_id", lambda x: x == 1) \
        .isComplete("job_title")  \
        .isComplete("max_salary") \
        .isComplete("min_salary") \
        .isUnique("job_id")  \
        .isNonNegative("max_salary") \
        .isNonNegative("min_salary")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

NameError: name 'spark' is not defined

In [10]:
checkResult_pandas_df = VerificationResult.successMetricsAsDataFrame(spark, checkResult, pandas=True)
checkResult_pandas_df

NameError: name 'spark' is not defined